# Outlook 2016 Email Examples Using Python

**Dependencies:**
- win32com library to interface with Microsoft Office products
    - Install with conda at DOS prompt> conda install pywin32

**Optional Dependencies:**
- pandas

**Additional References:**
- Microsoft's Outlook COM [reference](https://msdn.microsoft.com/en-us/library/microsoft.office.interop.outlook.mailitem_properties.aspx)

**NOTE:** Script uses local Outlook account

# Sending Email

In [ ]:
import win32com.client as win32

outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = "email1@some_company.com; email2@some_company.com"
mail.Subject = 'Your email subject'
mail.HTMLBody = "This email was automatically sent via Python Script"
mail.Send()

# Reading last email

StackOverflow [example](http://stackoverflow.com/questions/31619012/extract-senders-email-address-from-outlook-exchange-in-python-using-win32) that explains for the need to check the message Class for getting the right email format.

In [ ]:
import win32com.client

outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

inbox = outlook.GetDefaultFolder(6) # "6" refers to the index of a folder - in this case,
                                    # the inbox. You can change that number to reference
                                    # any other folder
all_inbox = inbox.Items
message = all_inbox.GetLast()

print("### Email Subject  ##")
print(message.subject, '\n')

print("###  Email Body  ###")
print(message.body)

print("###  Date Email Sent  ###")
print(message.SentOn, '\n')

print("###  Date Email Received  ###")
print(message.ReceivedTime, '\n')

print("###  Sender's First and Last Name  ###")
print(message.sender, '\n')

if message.Class == 43:
    if message.senderemailtype=='EX':
        print("###  Sender's email  ###")
        print(message.Sender.GetExchangeUser().PrimarySmtpAddress)
    else:
        print("###  Sender's email  ###")
        print(message.SenderEmailAddress)

# To get list of top email senders:

In [ ]:
import pandas as pd
from win32com.client import Dispatch
outlook = Dispatch("Outlook.Application").GetNamespace("MAPI")
inbox = outlook.GetDefaultFolder("6")
all_inbox = inbox.Items

df = pd.DataFrame()  # Create empty dataframe that we will be populating in the FOR loop
for msg in all_inbox:
    if msg.Class==43:
        if msg.SenderEmailType=='EX':
            # The format of the date stored in Outlook is giving errors, so forcing it to str format
            df = df.append({'SENDER':msg.Sender.GetExchangeUser().PrimarySmtpAddress, 
                            'DATE':str(msg.ReceivedTime)}, ignore_index=True)
        else:
            # The format of the date stored in Outlook is giving errors, so forcing it to str format
            df = df.append({'SENDER':msg.SenderEmailAddress, 'DATE':str(msg.ReceivedTime)}, ignore_index=True)

#### Because Outlook's date format is a non-standard format, it was giving errors.  As a workaround, it was stored as an object or string.  Then later, we will change the data type to date format.

In [ ]:
df.dtypes

In [ ]:
df.head()

### Change the data type of the ```DATE``` column to an actual date type so that we can query it as if it was a date:

In [ ]:
df.DATE = pd.to_datetime(df.DATE)

#### Note the ```DATE``` column is now a date format

In [ ]:
df.dtypes

In [ ]:
df.head()

### Finally, here is list of top email senders using value_counts() function:

In [ ]:
df.SENDER.value_counts()

#### No surprise notifications@yammer.com sends the most emails to me :-)

# Filter inbox emails by received date or by Sender

### For example, to get emails received between 2017-03-21 and 2017-03-22:

In [ ]:
filter_by_date = df.query("DATE >= '2017-03-21' and DATE <= '2017-03-22'")

In [ ]:
filter_by_date

### To get list of emails from a specific sender:

In [ ]:
filter_by_email = df.query("SENDER == 'notifications@yammer.com'")

In [ ]:
filter_by_email

## To obtain list of other Outlook folders and other useful objects:

In [ ]:
from win32com.client import Dispatch

outlook=Dispatch("Outlook.Application").GetNamespace("MAPI")
for i in range(50):
    try:
        box = outlook.GetDefaultFolder(i)
        name = box.Name
        print(i, name)
    except:
        pass

To summarize sent emails, from above, we can re-use the previous script by referencing "5" in the following:

In [ ]:
outlook.GetDefaultFolder("5")   # to access "Sent" folder

In [ ]:
import pandas as pd
from win32com.client import Dispatch
outlook = Dispatch("Outlook.Application").GetNamespace("MAPI")
sentbox = outlook.GetDefaultFolder("5")
all_sentbox = sentbox.Items

df = pd.DataFrame()  # Create empty dataframe that we will be populating in the FOR loop
for msg in all_sentbox:
    for recipient in msg.Recipients:
        print(recipient.address)
        
for msg in all_inbox:
    if msg.Class==43:
        if msg.SenderEmailType=='EX':
            # The format of the date stored in Outlook is giving errors, so forcing it to str format
            df = df.append({'SENDER':msg.Sender.GetExchangeUser().PrimarySmtpAddress, 
                            'DATE':str(msg.ReceivedTime)}, ignore_index=True)
        else:
            # The format of the date stored in Outlook is giving errors, so forcing it to str format
            df = df.append({'SENDER':msg.SenderEmailAddress, 'DATE':str(msg.ReceivedTime)}, ignore_index=True)

In [ ]:
import pandas as pd
from win32com.client import Dispatch
outlook = Dispatch("Outlook.Application").GetNamespace("MAPI")
sentbox = outlook.GetDefaultFolder("5")
all_sentbox = sentbox.Items

df = pd.DataFrame()  # Create empty dataframe that we will be populating in the FOR loop
        
for msg in all_sentbox:
    if msg.Class==43:
        if msg.SenderEmailType=='EX':
            print('Sender: ', msg.Sender.GetExchangeUser().PrimarySmtpAddress)
            for recipient in msg.Recipients:
                print(recipient.GetExchangeUser().PrimarySmtpAddress)
        else:
            for recipient in msg.Recipients:
                print(recipient.address)